# Lesson 3 Exercise 4: Using the WHERE Clause
![](../images/cassandralogo.png)

In this exercise we are going to walk through the basics of using the WHERE clause in Apache Cassandra. ##### denotes where the code needs to be completed.

In [ ]:
from src.database import (
    get_cs_cluster,
    get_cs_session,
    close_cs_session,
    shutdown_cs_cluster,
    insert_cs_rows,
    create_cs_keyspace,
    create_cs_table,
    drop_cs_table,
    drop_cs_keyspace,
    set_cs_keyspace,
)

cluster = get_cs_cluster()
session = get_cs_session(cluster)
keyspace_name = "my_keyspace"
drop_cs_keyspace(session, keyspace_name)  # drop any existing keyspace (nice for reruns)
create_cs_keyspace(session, keyspace_name)
set_cs_keyspace(session, keyspace_name)  # set the keyspace to the one we just created
tables = set()

Let's imagine we would like to start creating a new Music Library of albums. We want to ask 4 question of our data:
1. Give me every album in my music library that was released in a 1965 year
2. Give me the album that is in my music library that was released in 1965 by "The Beatles"
3. Give me all the albums released in a given year that was made in London 
4. Give me the city that the album "Rubber Soul" was recorded

# Data
![](images/table3.png)

How should we model this data? What should be our Primary Key and Partition Key? Since our data is looking for the YEAR let's start with that. From there we will add clustering columns on Artist Name and Album Name.

In [ ]:
# Drop and recreate keyspace to ensure clean schema
table_name_alb = "album_library"
tables.add(table_name_alb)
drop_cs_table(session, table_name_alb)  # drop the table if it exists
column_names = [
    "year",
    "artist_name",
    "album_name",
    "city",
]
column_types = [
    "int",
    "text",
    "text",
    "text",
]
columns_alb = dict(zip(column_names, column_types))
primary_keys = ["year", "artist_name", "album_name"]
create_cs_table(
    session,
    table_name_alb,
    columns_alb,
    primary_keys,
)

### Let's insert our data into of table

In [ ]:
insert_cs_rows(
    session,
    table_name_alb,
    list(columns_alb.keys()),
    [
        (1970, "The Beatles", "Let It Be", "Liverpool"),
        (1965, "The Beatles", "Rubber Soul", "Oxford"),
        (1965, "The Who", "My Generation", "London"),
        (1966, "The Monkees", "The Monkees", "Los Angeles"),
        (1970, "The Carpenters", "Close To You", "San Diego"),
    ],
)

### Validating our Data Model

Query 1

In [ ]:
# every album in my music library that was released in a 1965 year
query = "SELECT * FROM album_library WHERE year=1965"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.year, row.artist_name, row.album_name, row.city)

Query 2: 

In [ ]:
# the album that is in my music library that was released in 1965 by "The Beatles"
query = "SELECT * FROM album_library WHERE year=1965 AND artist_name='The Beatles'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.year, row.artist_name, row.album_name, row.city)

Query 3: 

In [ ]:
# all the albums released in a given year that was made in London
query = "SELECT * FROM album_library WHERE year=1965 AND city='London'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.year, row.artist_name, row.album_name, row.city)

Did you get an error? You can not try to access a column or a clustering column if you have not used the other defined clustering column. Let's see if we can try it a different way. 
Try Query 4: 



In [ ]:
# the city that the album "Rubber Soul" was recorded
query = "SELECT city FROM album_library WHERE year=1965 AND artist_name='The Beatles' AND album_name='Rubber Soul'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.city)

### Close session and drop tables

In [ ]:
for table in tables:
    drop_cs_table(session, table)
close_cs_session(session)
shutdown_cs_cluster(cluster)